In [95]:
import networkx as nx
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import pairwise_distances, silhouette_samples, silhouette_score

import numpy as np
import pandas as pd
from datetime import datetime

from collections import Counter
import itertools

from math import comb, ceil
import random
import string

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import multiprocessing as mp

import pickle

import sys
sys.path.insert(1, '../scripts')
import modeling_v3

import importlib
importlib.reload(modeling_v3)

import matplotlib.ticker as plticker
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 14

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
sim_no = 0
repetition = 64
random_seed = np.random.randint(100000)
n_beliefs = 6
m = 6
gamma = .8
n_nodes_to_connect = 1
n_agents = 2
last_step = 2000
is_composed = False

results = modeling_v3.simulate_multiple_agents(sim_no, random_seed, n_beliefs, m, gamma, n_nodes_to_connect, n_agents, last_step, is_composed)

100%|██████████| 2001/2001 [00:11<00:00, 177.92it/s]


In [96]:
repetition = 60
random_seed = np.random.randint(100000)
n_beliefs = 6
m = 15
gamma_vals = [0, .2, .4, .6, .8]
n_nodes_to_connect = 1
n_agents = 2
last_step = 8000
is_composed = False

results = {}

for gamma in gamma_vals:
    random_states = [np.random.randint(0, 10000000) for i in range(repetition)]    

    params = [*zip([*range(repetition)], 
                    random_states,
                    [n_beliefs] * repetition, 
                    [m] * repetition,
                    [gamma] * repetition,
                    [n_nodes_to_connect] * repetition,
                    [n_agents] * repetition,
                    [last_step] * repetition,
                    [is_composed] * repetition)]

    pool = mp.Pool(processes=30)
    temp = pool.starmap(modeling_v3.simulate_multiple_agents, params, chunksize=1)
    pool.close()

    results[gamma] = temp
    del temp

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
with open(f'../data/results_two_agents_{dt_string}.pkl', 'wb') as f:
    pickle.dump(results, f)

 98%|█████████▊| 7841/8001 [04:05<00:04, 38.14it/s] 

100%|█████████▉| 7980/8001 [04:14<00:00, 40.73it/s]

 97%|█████████▋| 7761/8001 [04:04<00:08, 28.01it/s]

100%|██████████| 8001/8001 [04:12<00:00, 31.65it/s]

100%|██████████| 8001/8001 [04:07<00:00, 32.32it/s]

100%|██████████| 8001/8001 [04:10<00:00, 31.95it/s] 

100%|██████████| 8001/8001 [04:05<00:00, 32.57it/s] 


In [98]:
results[.8][26]['track'][8000]['association_matrix'][0]

,+A,-A,+B,-B,+C,-C,+D,-D,+E,-E,+F,-F
+A,0,0,1,1,1,1,1,1,1,924,1,602
-A,0,0,1,1,1,1,1,4,53,21,13,38
+B,1,1,0,0,1,1,1,1,1,1,1,1
-B,1,1,0,0,1,1,1,1,3,1,26,982
+C,1,1,1,1,0,0,1,1354,1,1,1,1
-C,1,1,1,1,0,0,1,2,1,1,2,1
+D,1,1,1,1,1,1,0,0,1,1,1,1
-D,1,4,1,1,1354,2,0,0,3,1,1,1
+E,1,53,1,3,1,1,1,3,0,0,1,1
-E,924,21,1,1,1,1,1,1,0,0,1,1


In [120]:
results[.6][35]['track'][8000]

{'count_unique_belief_networks': 2,
 'pairwise_belief_sim': array([-1.]),
 'interpretative_dist': 0.058606300163950714,
 'internal_energies': [-1.0, -1.0],
 'belief_networks': array([[ 1,  1, -1, -1,  1,  1],
        [-1, -1,  1,  1, -1, -1]]),
 'association_matrix': {0:     +A   -A  +B   -B   +C  -C   +D  -D  +E   -E  +F   -F
  +A   0    0   1    1    1   1    1   1   1    1   1    1
  -A   0    0   1    3    1   2  376   7  19  533   1  571
  +B   1    1   0    0    1   1    1   1   1    1   1    1
  -B   1    3   0    0    1   1  825  16   1    1   1    1
  +C   1    1   1    1    0   0  460   1   1  633   1    1
  -C   1    2   1    1    0   0   23  10  27   32   1    1
  +D   1  376   1  825  460  23    0   0   1    1   1  488
  -D   1    7   1   16    1  10    0   0   2    1   1   10
  +E   1   19   1    1    1  27    1   2   0    0   1    2
  -E   1  533   1    1  633  32    1   1   0    0   1    1
  +F   1    1   1    1    1   1    1   1   1    1   0    0
  -F   1  571   1    1

In [94]:
modeling_v3.determine_relationships_likelihoods(results[.8][26]['track'][8000]['association_matrix'][0], gamma=.4)

({'A': {'B': 1, 'C': -1, 'D': -1, 'E': -1, 'F': 1, 'A': 0},
  'B': {'A': 1, 'C': -1, 'D': -1, 'E': -1, 'F': 1, 'B': 0},
  'C': {'A': -1, 'B': -1, 'D': 1, 'E': 1, 'F': -1, 'C': 0},
  'D': {'A': -1, 'B': -1, 'C': 1, 'E': 1, 'F': -1, 'D': 0},
  'E': {'A': -1, 'B': -1, 'C': 1, 'D': 1, 'F': -1, 'E': 0},
  'F': {'A': 1, 'B': 1, 'C': -1, 'D': -1, 'E': -1, 'F': 0}},
 {'A': {'B': None, 'C': None, 'D': None, 'E': None, 'F': None},
  'B': {'A': None, 'C': None, 'D': None, 'E': None, 'F': None},
  'C': {'A': None, 'B': None, 'D': None, 'E': None, 'F': None},
  'D': {'A': None, 'B': None, 'C': None, 'E': None, 'F': None},
  'E': {'A': None, 'B': None, 'C': None, 'D': None, 'F': None},
  'F': {'A': None, 'B': None, 'C': None, 'D': None, 'E': None}})

In [86]:
results[1][30]['track'][4000]

{'count_unique_belief_networks': 2,
 'pairwise_belief_sim': array([0.26320442]),
 'interpretative_dist': 0.4386763858905969,
 'internal_energies': [-1.0, -1.0],
 'belief_networks': array([[-1.        ,  0.3498831 , -0.93378777, -1.        , -0.76610624,
          0.53564519],
        [-1.        , -0.41560231,  0.59730635, -1.        ,  0.31848946,
         -0.35985977]]),
 'association_matrix': {0:     +A   -A  +B   -B   +C  -C  +D   -D   +E  -E  +F   -F
  +A   0    0   1    1    1   1   1    1    1   1   1    1
  -A   0    0   1   65   67   1   1  647   63   1   1   76
  +B   1    1   0    0    1   1   1    1    1   1   1    1
  -B   1   65   0    0  131   1   1   74  137   1   1  141
  +C   1   67   1  131    0   0   1   79  119   1   1  136
  -C   1    1   1    1    0   0   1    1    1   1   1    1
  +D   1    1   1    1    1   1   0    0    1   1   1    1
  -D   1  647   1   74   79   1   0    0   60   1   1   70
  +E   1   63   1  137  119   1   1   60    0   0   1  123
  -E   1 

In [62]:
temp

[1, 4, 5, 6]

In [166]:
association_matrix = results[0.2][6]['track'][4000]['association_matrix'][0]

In [152]:
propositions = set([c[1] for c in association_matrix.columns])

In [158]:
pairs = [*itertools.combinations(propositions,2)]

In [167]:
compressed_matrix = association_matrix.groupby(lambda x: x[1], axis=1).sum().groupby(lambda x: x[1], axis=0).sum()

In [172]:
relationships, likelihoods = modeling_v3.determine_relationships_likelihoods(association_matrix, gamma)

In [173]:
relationships

{'A': {'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 1},
 'B': {'A': 0, 'C': 0, 'D': 0, 'E': -1, 'F': 0},
 'C': {'A': 0, 'B': 1, 'D': 0, 'E': 0, 'F': 0},
 'D': {'A': 0, 'B': 1, 'C': 0, 'E': 0, 'F': 0},
 'E': {'A': 0, 'B': -1, 'C': 0, 'D': 0, 'F': 0},
 'F': {'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0}}